In [2]:
import pandas as pd
import re
from tqdm.notebook import tqdm
import time
from datetime import datetime

In [3]:
df = pd.read_csv("reestr_17_1831.csv",encoding='utf-8',sep=';')

In [5]:
df.columns

Index(['населенный_пункт', 'тип_тс', 'тип_маршрута', 'номер_маршрута',
       'наим_маршрута', 'маршут_след_прям', 'маршут_след_обрат',
       'список_оп_прям', 'список_оп_обрат', 'режим_работы', 'интервалы',
       'выпуск', 'время_в_пути', 'протяженность', 'предприятие', 'подв_состав',
       'стоимость_проезда', 'место_отстоя', 'место_отправл', 'время_отправл',
       'интерв_ср_буд', 'интерв_ср_вых', 'интерв_ср_вчр', 'пнд', 'втр', 'срд',
       'чтв', 'птн', 'сбт', 'вск', 'ссылка'],
      dtype='object')

In [123]:
lst1 = list(df.интервалы.unique())

In [55]:
def fnd_tm_strt_end(string):
    
    lst_dgt = re.findall(r'\d+', string)
    if len(lst_dgt) > 1:
        
        l1 = []
        for dgt in lst_dgt:
            l1.append(int(dgt))
        #
        l1.sort()
        tm_strt = min(l1)
        tm_end = max(l1)
        cnt=0
        while ((tm_strt < 5) & (tm_strt < tm_end)):
            try:
                tm_strt = min(l1[cnt:])
            except:
                tm_strt = 6
                break
            cnt+=1
        #
        cnt=len(l1)
        while ((tm_end > 23) & (tm_strt < tm_end)):
            try:
                tm_end = max(l1[:cnt])
            except:
                tm_end = 22
                break
            cnt-=1
        #
    #
    else:
        tm_strt = 6
        tm_end = 22
    #
    return tm_strt, tm_end

In [116]:
lst_reg_rab = list(df.режим_работы)

lst_strt_wrk,lst_end_wrk,lst_strt_hld,lst_end_hld = [],[],[],[]

i=0
for i in range(len(lst_reg_rab)):
    string = str(lst_reg_rab[i])
    if string == 'nan':
        tm_strt_wrk, tm_end_wrk = '-','-'
        tm_strt_hld, tm_end_hld = '-','-'
    #
    elif 'жедневно' in string:
        tm_strt_wrk, tm_end_wrk = fnd_tm_strt_end(string)
        tm_strt_hld, tm_end_hld = tm_strt_wrk, tm_end_wrk
    #
    elif 'рабочи' in string:
        ind_hld = string.find('выходны')
        str_wrk = string[:ind_hld]
        tm_strt_wrk, tm_end_wrk = fnd_tm_strt_end(str_wrk)
        if 'выходны' in string:
            str_hld = string[ind_hld:]
            tm_strt_hld, tm_end_hld = fnd_tm_strt_end(str_hld)
        else:
            tm_strt_hld, tm_end_hld = '-','-'
    #
    elif 'выходны' in string:
        if 'рабочи' in string:
            continue
        else:
            tm_strt_wrk, tm_end_wrk = '-','-'
            tm_strt_hld, tm_end_hld = fnd_tm_strt_end(string)
    #
    else:
        tm_strt_wrk, tm_end_wrk = fnd_tm_strt_end(string)
        tm_strt_hld, tm_end_hld = tm_strt_wrk, tm_end_wrk
    #
    def mk_tm(one):
        if one != '-':
            one = str(one) + ':00'
        return one
    # 
    lst_strt_wrk.append(mk_tm(tm_strt_wrk))
    lst_end_wrk.append(mk_tm(tm_end_wrk))
    lst_strt_hld.append(mk_tm(tm_strt_hld))
    lst_end_hld.append(mk_tm(tm_end_hld))
# 


In [117]:
print(len(lst_strt_wrk),len(lst_end_wrk),len(lst_strt_hld),len(lst_end_hld))

4073 4073 4073 4073


In [120]:
# data1=list(zip(lst_reg_rab,lst_strt_wrk,lst_end_wrk,lst_strt_hld,lst_end_hld))
# columns1=['string','wrk_strt','wrk_end','hld_strt','hld_end']

# tmp_df = pd.DataFrame(data=data1,columns=columns1)

In [124]:
lst1 = list(df.интервалы.unique())

In [ ]:
data2 = list(zip('wrk_min','wrk_max','hld_min','hld_max',
                 'evn_wrk_min','evn_wrk_max','evn_hld_min','evn_hld_max'))

In [ ]:
lst_wrk_min,lst_wrk_max,lst_hld_min,lst_hld_max = [],[],[],[]
lst_evn_wrk_min,lst_evn_wrk_max,lst_evn_hld_min,lst_evn_hld_max = [],[],[],[]

In [177]:
def fnd_interv(string):
    ind_sep = string.find(':')
    if ind_sep == -1:
        ind_sep = 0
    #
    ind_mnt = string.find('мин')
    str_part = string[ind_sep:ind_mnt]

    lst_dgt = re.findall(r'\d+', str_part)
    if len(lst_dgt) > 0:

        l1 = []
        for dgt in lst_dgt:
            l1.append(int(dgt))
        #
        l1.sort()
        min_interv = min(l1)
        max_interv = max(l1)
    #
    else:
        min_interv = 1
        max_interv = 180
    #

    return min_interv, max_interv

In [183]:
def fnd_day_tp(string,tp,rplc_min,replc_max):
    if tp in string:
        ind_tp = string.find(tp)
        if tp == 'мин':
            str_tp = string[:ind_tp]
        else:
            str_tp = string[ind_tp:]
        min_res,max_res = fnd_interv(str_tp)
    else:
        min_res,max_res = rplc_min,replc_max
    #
    return min_res,max_res

In [190]:
lst_wrk_min,lst_wrk_max,lst_hld_min,lst_hld_max = [],[],[],[]
lst_evn_wrk_min,lst_evn_wrk_max,lst_evn_hld_min,lst_evn_hld_max = [],[],[],[]

i=0
for i in range(len(lst1)):
    string=str(lst1[i])
    if ((string == 'nan') | ('расписани' in string)):
        wrk_min,wrk_max,hld_min,hld_max = '-','-','-','-'
        evn_wrk_min,evn_wrk_max,evn_hld_min,evn_hld_max = '-','-','-','-'
    elif (('рабочи' in string) | ('выходн' in string) | ('вечерн' in string)):
        wrk_min,wrk_max = fnd_day_tp(string,'рабочи',0,0)
        if ((wrk_min==0)&(wrk_max == 0)):
            wrk_min,wrk_max = fnd_day_tp(string,'мин',0,0)
        #
        hld_min,hld_max = fnd_day_tp(string,'выходн',wrk_min,wrk_max)
        evn_wrk_min,evn_wrk_max = fnd_day_tp(string,'вечерн',wrk_min,wrk_max)
    #
    elif 'мин' in string:
        wrk_min,wrk_max = fnd_interv(string)
        hld_min,hld_max,evn_wrk_min,evn_wrk_max = wrk_min,wrk_max,wrk_min,wrk_max
    else:
        wrk_min,wrk_max,hld_min,hld_max = 0,0,0,0
        evn_wrk_min,evn_wrk_max,evn_hld_min,evn_hld_max = 0,0,0,0
    #
    evn_hld_min,evn_hld_max = evn_wrk_min,evn_wrk_max
    
    lst_value=[wrk_min,wrk_max,hld_min,hld_max,
               evn_wrk_min,evn_wrk_max,evn_hld_min,evn_hld_max]
    lst_lsts=[lst_wrk_min,lst_wrk_max,lst_hld_min,lst_hld_max,
             lst_evn_wrk_min,lst_evn_wrk_max,lst_evn_hld_min,lst_evn_hld_max]
    j=0
    for j in range(len(lst_value)):
        lst_lsts[j].append(lst_value[j])
#